In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx

In [2]:
equity_list = ['DELL', 'IBM', 'MSFT', 'ORCL']
alpha_list = [-0.05, 0.07, 0.04, -0.02]
alpha_series = pd.Series(index=equity_list, data=alpha_list)
cov_list = [
    [0.02, 0.0001, 0.002, -0.0001],
    [0.0001, 0.04, 0.0003, 0.0009],
    [0.002, 0.0003, 0.05, 0.0001],
    [-0.0001, 0.0009, 0.0001, 0.02]
]
cov_df = pd.DataFrame(index=equity_list, columns=equity_list, data=cov_list)

In [3]:
print(alpha_series)
print('\n')
print(cov_df)

DELL   -0.05
IBM     0.07
MSFT    0.04
ORCL   -0.02
dtype: float64


        DELL     IBM    MSFT    ORCL
DELL  0.0200  0.0001  0.0020 -0.0001
IBM   0.0001  0.0400  0.0003  0.0009
MSFT  0.0020  0.0003  0.0500  0.0001
ORCL -0.0001  0.0009  0.0001  0.0200


In [4]:
reference_size = 1.
risk_avn = 1. / reference_size

# 约束条件

In [5]:
ub = pd.Series()
lb = pd.Series()
group_b = {}

In [6]:
pre_holding = pd.Series(index=equity_list, data=0.)
pre_holding['cash'] = reference_size
current_price = pd.Series(index=equity_list+['cash'], data=1.)

# 输入

In [7]:
# alpha和cov扩展ｃａｓｈ
alpha_series['cash'] = 0.
cov_df = cov_df.reindex(equity_list + ['cash']).fillna(0.)
cov_df['cash'] = 0.
print(alpha_series)
print('\n')
print(cov_df)
trading_day = '20100101'
cov_panel = pd.Panel(
    data={trading_day: cov_df})
annual_rt_df = pd.DataFrame(
    index=[trading_day],
    data=[alpha_series])
# benchmark
benchmark_weight = pd.Series(index=equity_list+['cash'], data=[0.3, 0.23, 0.12, 0.35, 0.])

DELL   -0.05
IBM     0.07
MSFT    0.04
ORCL   -0.02
cash    0.00
dtype: float64


        DELL     IBM    MSFT    ORCL  cash
DELL  0.0200  0.0001  0.0020 -0.0001   0.0
IBM   0.0001  0.0400  0.0003  0.0009   0.0
MSFT  0.0020  0.0003  0.0500  0.0001   0.0
ORCL -0.0001  0.0009  0.0001  0.0200   0.0
cash  0.0000  0.0000  0.0000  0.0000   0.0


/Users/Casey/anaconda2/envs/gluon/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3325: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


# CVXPY配置

In [8]:
from solar.cvxportfolio.constraints import LongCash, LongOnly
from solar.cvxportfolio.costs import TcostModel
from solar.cvxportfolio.returns import ReturnsForecast
from solar.solver.components.constraints import AssetWeightBound, TurnoverLimit, GroupWeightBound
from solar.solver.components.policies import MaxUtilityPolicy
from solar.solver.components.risks import AssetSigma
from solar.math.common_function import adjust_holding_precision
solver_param = {}
solver_param.update({
    cvx.ECOS: {
        'abstol': 1.0e-09,
        'reltol': 1.0e-09,
        'max_iters': 150
    }
})

In [9]:
class MaxUtilityFunctionSolver:
    def __init__(self, param):
        self.asset_list = param['asset_list']
        self.asset_cov = param['asset_cov']
        self.asset_rt = param['asset_rt']
        self.risk_aversion = param['risk_aversion']
        self.total_weight = param['total_weight']
        self.target_turnover = param['target_turnover']
        self.fee = param['fee']
        self.lower_boundary = param['lb']
        self.upper_boundary = param['ub']
        self.group_boundary = param['group_b']
        self.trading_day = param['trading_day']
        self.pre_holding = param['pre_holding']
        self.current_price = param['current_price']
        self.solver = param['solver']
        self.weight_precision = param['weight_precision']
        self.benchmark_weight_series = param['benchmark_weight_series']
        self.is_long_only = param['is_long_only']
        self.is_long_cash = param['is_long_cash']
        self.lower_boundary = self.lower_boundary.reindex(self.asset_list).fillna(-np.inf)
        self.upper_boundary = self.upper_boundary.reindex(self.asset_list).fillna(np.inf)

    def solve(self):
        transaction_cost_mdl = TcostModel(self.fee) if self.fee is not None else None
        cost_models = []
        if transaction_cost_mdl is not None:
            cost_models.append(transaction_cost_mdl)

        asset_cov_mdl = AssetSigma(self.asset_cov, w_bench=self.benchmark_weight_series)
        return_mdl = ReturnsForecast(self.asset_rt)
        weight_bound_list = [x for x in zip(self.lower_boundary, self.upper_boundary)]
        if self.total_weight:
            weight_bound_list += [1 - self.total_weight]
        wt_cons = AssetWeightBound(weight_bound_list)
        group_wt_cons = GroupWeightBound(self.group_boundary)
        constraints = []
        if self.is_long_only:
            constraints += [LongOnly()]
        if self.is_long_cash:
            constraints += [LongCash()]
        constraints = [wt_cons, group_wt_cons]

        if self.target_turnover is not None:
            turnover_cons = TurnoverLimit(self.target_turnover)
            constraints.append(turnover_cons)

        policy = MaxUtilityPolicy(
            return_mdl,
            asset_cov_mdl,
            self.risk_aversion,
            costs=cost_models,
            constraints=constraints,
            solver=self.solver,
            solver_opts=solver_param[self.solver])
        prob, trade_value = policy.get_trades(self.pre_holding, self.trading_day)
        trade_volume = trade_value.drop('cash') / self.current_price
        trade_volume.fillna(0., inplace=True)
        after_trade_value = self.pre_holding + trade_value
        after_trade_ratio = after_trade_value / after_trade_value.sum()
        after_trade_ratio = adjust_holding_precision(after_trade_ratio, self.weight_precision)
        transaction_cost_sum = 0.
        for cost in policy.costs:
            if isinstance(cost, TcostModel):
                transaction_cost_sum += cost.value_expr(0., 0., trade_value)
        trade_result = {
            'trade_value': trade_value,
            'trade_volume': trade_volume,
            'solution_status': prob.status,
            'transaction_cost_sum': transaction_cost_sum,
            'weight': after_trade_ratio,
            'holding': after_trade_value
        }
        return trade_result


In [10]:
free_param = {'asset_list': equity_list,
         'asset_cov': cov_panel,
         'asset_rt': annual_rt_df,
         'risk_aversion': risk_avn,
         'total_weight': None,
         'target_turnover': None,
         'fee': None,
         'lb': lb,
         'ub': ub,
         'group_b': group_b,
         'trading_day': trading_day,
         'pre_holding': pre_holding,
         'current_price': current_price,
         'solver': cvx.ECOS,
         'weight_precision': 0.001,
         'benchmark_weight_series': benchmark_weight,
         'is_long_only': False,
         'is_long_cash': False
        }

In [11]:
model = MaxUtilityFunctionSolver(free_param)
free_result = model.solve()

/Users/Casey/anaconda2/envs/gluon/lib/python3.6/site-packages/pandas/core/dtypes/missing.py:106: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  return _isna(obj)


In [13]:
free_result['holding']

DELL   -2.303920
IBM     2.004482
MSFT    1.015705
ORCL   -0.747341
cash    1.031074
dtype: float64

In [14]:
ub = pd.Series(index=equity_list, data=[np.inf, 0.5, np.inf, np.inf])
lb = pd.Series(index=equity_list, data=-0.05)
total_weight = 1.
group_b = {}
constraint_param = {'asset_list': equity_list,
         'asset_cov': cov_panel,
         'asset_rt': annual_rt_df,
         'risk_aversion': risk_avn,
         'total_weight': total_weight,
         'target_turnover': None,
         'fee': None,
         'lb': lb,
         'ub': ub,
         'group_b': group_b,
         'trading_day': trading_day,
         'pre_holding': pre_holding,
         'current_price': current_price,
         'solver': cvx.ECOS,
         'weight_precision': 0.001,
         'benchmark_weight_series': benchmark_weight,
         'is_long_only': False,
         'is_long_cash': False
        }

In [15]:
model = MaxUtilityFunctionSolver(constraint_param)
constraint_result = model.solve()

In [16]:
constraint_result['holding']

DELL   -5.000000e-02
IBM     5.000000e-01
MSFT    6.000000e-01
ORCL   -5.000000e-02
cash    4.984901e-14
dtype: float64

# alpha decomposition

个股权重设置了下限为$-5\%$，IBM的上限为$50\%$，总权重为$1$，共6个约束条件，其中有5个为不等式约束，1个为等式约束。
原目标函数为
$$
\max \alpha^T x - \frac{1}{2} (x-b)^T Q (x-b)
$$
将约束条件带入原目标函数中，利用拉格朗日乘子法，得到拉格朗日函数
$$
L(x, \lambda_{1-6})= \alpha^T x - \frac{1}{2} (x-b)^T Q (x-b) - \lambda_1(\sum_{i=2}^5 x_i - 1) - \sum_{i=2}^5(\lambda_i(-0.05-x_i)) - \lambda_6(x_2-0.5)
$$
其中，乘子$\lambda_1$对应的是等式约束，$\lambda_{2-6}$对应的是不等式约束。

拉格朗日函数$L$对变量$x$求导，可得
$$
\frac{\partial L}{\partial x} = \alpha - Q^T(x-b) - \lambda_1 I + [\lambda_2,\lambda_3,\lambda_4,\lambda_5]^T - [0,\lambda_6,0,0]^T
$$
令偏导数等于0，整理两边可得
$$
\alpha - Q^T(x^*-b) = 
\left[
  \begin{matrix}
   \lambda_1^*-\lambda_2^*\\
   \lambda_1^*-\lambda_3^*-\lambda_6^*\\
   \lambda_1^*-\lambda_4^*\\
   \lambda_1^*-\lambda_5^*\\
  \end{matrix}
\right]
$$


不等式约束需要满足根据KKT条件，这里只列出相关的条件
$$
\lambda_2^*(-0.05-x_1^*)=0 \\
\lambda_3^*(-0.05-x_2^*)=0 \\
\lambda_4^*(-0.05-x_3^*)=0 \\
\lambda_5^*(-0.05-x_4^*)=0 \\
\lambda_6^*(x_2^*-0.5)=0 \\
$$

通过求解的结果可以看到，DELL、IBM、ORCL达到了约束条件的边界，它们分别的权重是

$w^*_{\text{DELL}}=-0.05,w^*_{\text{IBM}}=0.5,w^*_{\text{ORCL}}=-0.05$。

因此，这些约束对应的乘子$\lambda \neq 0$，对应的乘子有$\lambda_2^*,\lambda_6^*,\lambda_5^*$。因此其他不等式约束对应的乘子为$0$。这里有$\lambda_3^*=0,\lambda_4^*=0$。

联系偏导数得到的关于$\lambda$的等式，可以求的所有的$\lambda^*$。

In [17]:
# implied alpha
def implied_alpha(cov_df, optimal_weight, wb_bench):
    return cov_df.dot(optimal_weight - wb_bench)/reference_size
im_alpha = implied_alpha(cov_df, constraint_result['holding'], benchmark_weight)
im_alpha

DELL   -0.005973
IBM     0.010549
MSFT    0.023341
ORCL   -0.007674
cash    0.000000
dtype: float64

In [21]:
lm = alpha_series - im_alpha
lm

DELL   -0.044027
IBM     0.059451
MSFT    0.016659
ORCL   -0.012326
cash    0.000000
dtype: float64

In [31]:
# budget alpha
lm4 = 0
lm3 = 0
lm1 = lm.iloc[2] + lm4
lm2 = lm1 - lm.iloc[0]
lm5 = lm1 - lm.iloc[3]
lm6 = lm1 - lm.iloc[1] - lm3

In [36]:
lm6

-0.04279199999718952

$\lambda$和对应的约束条件相乘后，就是该约束条件对应的alpha分解的部分。以等式约束$\sum_i x_i=1$为例，alpha分解到这个部分是可以从偏$L$关于$x$的偏导数中看出，为$-\lambda_1 I=[-\lambda_1,-\lambda_1,-\lambda_1,-\lambda_1]=[-0.0167,-0.0167,-0.0167,-0.0167]$。

DELL下限被分解到的alpha为$[\lambda_2,0,0,0]=[0.0607,0,0,0]$

IBM下限被分解到的alpha为$[0,\lambda_3,0,0]=[0,0,0,0]$

MSFT下限被分解到的alpha为$[0,0,\lambda_4,0]=[0,0,0,0]$

ORCL下限被分解到的alpha为$[0,0,0,\lambda_5]=[0,0,0,0.0290]$

IBM上限被分解到的alpha为$[0,\lambda_6,0,0]=[0,-0.0427,0,0]$